In [1]:
# make sure you don't hog all the video memory
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)
from keras import backend as K
K.set_session(sess)
###################################

from keras.layers import Input, Lambda, merge, Dense, DepthwiseConv2D, Activation, AveragePooling2D
from keras.layers import Flatten,Conv2D, MaxPooling2D, Dropout, BatchNormalization, SeparableConv2D
from keras.constraints import max_norm
from keras.models import Model, Sequential
from keras.regularizers import l2
from keras.initializers import RandomUniform
from keras.optimizers import SGD,Adam
from keras.losses import binary_crossentropy
import numpy.random as rng
import numpy as np
import os
import pickle
import matplotlib.pyplot as plt
# import seaborn as sns
from sklearn.utils import shuffle
%matplotlib inline

Using TensorFlow backend.


In [2]:
file = 'Merged456-197-289_ICA(-eyes)+AUDpreproc.mat, DS2=64Hz, FIR=2-30Hz, centnorm=1, step=2, win=2, TD, 1-93.mat' #0.75(unstable)

# get the Dataset:
import scipy.io as sio
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

path = '/home/amplifier/home/DATASETS/' + file
mat_contents = sio.loadmat(path)
X = mat_contents['X']
Y = mat_contents['Z']

if X.shape[1]<X.shape[2]:
    X = np.transpose(X,[0,2,1])

if Y.shape[1] > Y.shape[0]:
    Y = Y.T

    
# # one hot encode the labels:
onehot_encoder = preprocessing.OneHotEncoder(sparse=False)
Y = onehot_encoder.fit_transform(Y)

X = X.transpose(0,2,1)

print('Original data shape:', X.shape)
print('Original labels shape:', Y.shape)

Original data shape: (450, 60, 128)
Original labels shape: (450, 2)


In [3]:
a,b = [],[]
for i in range(len(Y)):
    a.append(i if np.sum((Y[i]-np.array([1,0]))**2)==0 else None)
a = [x for x in a if x is not None]

for i in range(len(Y)):
    b.append(i if np.sum((Y[i]-np.array([0,1]))**2)==0 else None)
b = [x for x in b if x is not None]

dat = np.zeros([len(X),4])
print(dat.shape)
dat[:,0] = np.random.choice(a, len(X), replace=True)
dat[:,1] = np.random.choice(a, len(X), replace=True)
dat[:,2] = np.random.choice(b, len(X), replace=True)
dat[:,3] = 0 # a

dat1 = np.zeros([len(X),4])
dat1[:,0] = np.random.choice(b, len(X), replace=True)
dat1[:,1] = np.random.choice(b, len(X), replace=True)
dat1[:,2] = np.random.choice(a, len(X), replace=True)
dat1[:,3] = 1 # b

dat = np.vstack((dat,dat1))
print(dat.shape)

XX, YY = [], []
for i in range(len(dat)):
    XX.append([X[dat[i,0].astype(int),:,:], X[dat[i,1].astype(int),:,:], X[dat[i,2].astype(int),:,:]])
    YY.append(dat[i,3])

XX[1][1].shape
xx_train, xx_test, yy_train, yy_test = train_test_split(XX, YY, test_size=0.2, shuffle=True)

print('xx_train:', len(xx_train))
print('xx_test:', len(xx_test))

xx_train = np.asarray(xx_train)
xx_test = np.asarray(xx_test)

print('Transformed to numpy array:')
print('xx_train:', xx_train.shape)
print('xx_test:', xx_test.shape)

(450, 4)
(900, 4)
xx_train: 720
xx_test: 180
Transformed to numpy array:
xx_train: (720, 3, 60, 128)
xx_test: (180, 3, 60, 128)


In [4]:
def EEGNet_my(input1, nb_classes, Chans = 64, Samples = 128, 
             dropoutRate = 0.25, kernLength = 64, F1 = 4, 
             D = 2, F2 = 8, dropoutType = 'Dropout'):
    
    """ Keras Implementation of EEGNet (https://arxiv.org/abs/1611.08024)

    Inputs:
        
      nb_classes      : int, number of classes to classify
      Chans, Samples  : number of channels and time points in the EEG data
      dropoutRate     : dropout fraction
      kernLength      : length of temporal convolution in first layer. We found
                        that setting this to be half the sampling rate worked
                        well in practice. For the SMR dataset in particular
                        since the data was high-passed at 4Hz we used a kernel
                        length of 32.     
      F1, F2          : number of temporal filters (F1) and number of pointwise
                        filters (F2) to learn. Default: F1 = 4, F2 = F1 * D. 
      D               : number of spatial filters to learn within each temporal
                        convolution. Default: D = 2
      dropoutType     : Either SpatialDropout2D or Dropout, passed as a string.

    """
    
    if dropoutType == 'SpatialDropout2D':
        dropoutType = SpatialDropout2D
    elif dropoutType == 'Dropout':
        dropoutType = Dropout
    else:
        raise ValueError('dropoutType must be one of SpatialDropout2D '
                         'or Dropout, passed as a string.')
    
    init = RandomUniform(minval=-0.1, maxval=0.1, seed=29)
    net = Sequential()
    net.add (Conv2D(F1, (1, kernLength), padding = 'same',
                                   input_shape = (Chans, Samples,1),
                                   use_bias = False, bias_initializer=init, kernel_initializer=init))
    net.add (BatchNormalization())
    net.add (DepthwiseConv2D((Chans, 1), use_bias = False, 
                                   depth_multiplier = D,
                                   depthwise_constraint = max_norm(1.), bias_initializer=init, kernel_initializer=init))
    net.add (BatchNormalization())
    net.add (Activation('elu'))
    net.add (AveragePooling2D((1, 4)))
    net.add (dropoutType(dropoutRate))
    net.add (SeparableConv2D(F2, (1, 16), use_bias = False, padding = 'same', bias_initializer=init, kernel_initializer=init))
    net.add (BatchNormalization())
    net.add (Activation('elu'))
    net.add (AveragePooling2D((1, 8)))
    net.add (dropoutType(dropoutRate))
    net.add (Flatten())
    net.add (Dense(nb_classes, kernel_constraint = max_norm(0.25), bias_initializer=init, kernel_initializer=init))
    return net

In [5]:
xx_train.shape

(720, 3, 60, 128)

In [6]:
from keras.engine.topology import Layer

def triplet_loss(y_true, y_pred):
    norm1 = K.sqrt(K.sum(K.square(y_pred[0] - y_pred[1]), axis=-1, keepdims=True))
    norm2 = K.sqrt(K.sum(K.square(y_pred[0] - y_pred[2]), axis=-1, keepdims=True))
    loss = norm1 - norm2 + 0.2
    return loss

# input_shape = (64, 128, 1)
input_shape = xx_train[-1,-1,:,:].shape + (1,)

a_input = Input(input_shape)
r_input = Input(input_shape)
n_input = Input(input_shape)

#call the convnet Sequential model on each of the input tensors so params will be shared
# encoded NOT as Sequential (stack of layers), but as a Tensor!!!! if you add an argument, a Tensor is returned
encoded_a = EEGNet_my(input_shape, 10, Chans=input_shape[0])(a_input)
encoded_r = EEGNet_my(input_shape, 10, Chans=input_shape[0])(r_input)
encoded_n = EEGNet_my(input_shape, 10, Chans=input_shape[0])(n_input)

class MergeLegs(Layer):
    def __init__(self, **kwargs):
        super(MergeLegs, self).__init__(**kwargs)

    def call(self ,x ,mask=None):
        a = x[0]
        r = x[1]
        n = x[2]
        norm1 = K.sqrt(K.sum(K.square(a - r), axis=-1, keepdims=True))
        norm2 = K.sqrt(K.sum(K.square(a - n), axis=-1, keepdims=True))
        loss = norm1 - norm2 + 0.5
        self.add_loss(loss, x)
        #you can output whatever you need, just update output_shape adequately
        #But this is probably useful
        return K.concatenate([a,r,n], axis=1)

    def get_output_shape_for(self, input_shape):
        return (input_shape[0][0],1)
    
# prediction = MergeLegs()([encoded_a, encoded_r, encoded_n])

# siamese_net = Model(inputs=[a_input, r_input, n_input], outputs=prediction)
# siamese_net.summary()
# siamese_net.compile(loss=triplet_loss, optimizer='adam')

# # layer to merge two encoded inputs with the l1 distance between them
# # L1_layer = Lambda(lambda tensors:K.abs(tensors[0] + tensors[1] + tensors[2]))

L1_layer = Lambda(lambda tensors:
                  K.sqrt(K.sum(K.square(tensors[0] - tensors[1]), axis=-1, keepdims=True)) -
                  K.sqrt(K.sum(K.square(tensors[0] - tensors[2]), axis=-1, keepdims=True)) +
                  0.1)

# L1_layer = Lambda(lambda tensors:
#                   K.tanh(K.sqrt(K.sum(K.square(tensors[0] - tensors[1]), axis=-1, keepdims=True))) -
#                   K.tanh(K.sqrt(K.sum(K.square(tensors[0] - tensors[2]), axis=-1, keepdims=True))) +
#                   0.1)

#call this layer on list of three input tensors.
L1_distance = L1_layer([encoded_a, encoded_r, encoded_n])
prediction = Dense(1,activation='sigmoid')(L1_distance)

siamese_net = Model(inputs=[a_input, r_input, n_input], outputs=prediction)
siamese_net.compile(loss='binary_crossentropy', optimizer='adam')
siamese_net.summary()

In [8]:
# Train the Siamese model:
train_mock_labels = np.zeros(len(xx_train))
test_mock_labels = np.zeros(len(xx_test))

# Training time!
from keras.callbacks import EarlyStopping, ModelCheckpoint
early_stopping = EarlyStopping(monitor='val_loss', patience=100, mode='min')
checkpointer = ModelCheckpoint(filepath='/home/amplifier/home/NEW_DL/weights/Siamese.h5',
                               verbose=1,
                               monitor='val_acc',
                               save_best_only=True)

train_history = siamese_net.fit(x = [xx_train[:,0,:,:,None], xx_train[:,1,:,:,None], xx_train[:,2,:,:,None]],
                y = train_mock_labels,
                epochs=100,
                batch_size=20,
                verbose=2,
                shuffle=True,
                validation_data=([xx_test[:,0,:,:,None], xx_test[:,1,:,:,None], xx_test[:,2,:,:,None]], test_mock_labels),
                callbacks=[checkpointer, early_stopping])

# model.save('/home/amplifier/home/NEW_DL/models/EEGnet.h5')

Train on 720 samples, validate on 180 samples
Epoch 1/100
 - 4s - loss: 0.5664 - val_loss: 0.2932
Epoch 2/100


/opt/conda/lib/python3.6/site-packages/keras/callbacks.py:435: RuntimeWarning: Can save best model only with val_acc available, skipping.
  'skipping.' % (self.monitor), RuntimeWarning)


 - 1s - loss: 0.2104 - val_loss: 0.1221
Epoch 3/100
 - 1s - loss: 0.0722 - val_loss: 0.0684
Epoch 4/100
 - 1s - loss: 0.0305 - val_loss: 0.0387
Epoch 5/100
 - 1s - loss: 0.0162 - val_loss: 0.0223
Epoch 6/100
 - 1s - loss: 0.0109 - val_loss: 0.0138
Epoch 7/100
 - 1s - loss: 0.0075 - val_loss: 0.0092
Epoch 8/100
 - 1s - loss: 0.0057 - val_loss: 0.0066
Epoch 9/100
 - 1s - loss: 0.0043 - val_loss: 0.0047
Epoch 10/100
 - 1s - loss: 0.0036 - val_loss: 0.0036
Epoch 11/100
 - 1s - loss: 0.0028 - val_loss: 0.0028
Epoch 12/100
 - 1s - loss: 0.0023 - val_loss: 0.0023
Epoch 13/100
 - 1s - loss: 0.0021 - val_loss: 0.0019
Epoch 14/100
 - 1s - loss: 0.0018 - val_loss: 0.0016
Epoch 15/100
 - 1s - loss: 0.0017 - val_loss: 0.0013
Epoch 16/100
 - 1s - loss: 0.0014 - val_loss: 0.0011
Epoch 17/100
 - 1s - loss: 0.0013 - val_loss: 0.0010
Epoch 18/100
 - 1s - loss: 0.0011 - val_loss: 8.9380e-04
Epoch 19/100
 - 1s - loss: 0.0011 - val_loss: 7.9384e-04
Epoch 20/100
 - 1s - loss: 9.9048e-04 - val_loss: 7.0620e-

KeyboardInterrupt: 

In [67]:
testmod_1 = Sequential()
testmod_1.add(siamese_net.layers[0])
for i in range(14):
    testmod_1.add(siamese_net.layers[3].layers[i])
testmod_1.summary()

testmod_2 = Sequential()
testmod_2.add(siamese_net.layers[0])
for i in range(14):
    testmod_2.add(siamese_net.layers[4].layers[i])
    
testmod_3 = Sequential()
testmod_3.add(siamese_net.layers[0])
for i in range(14):
    testmod_3.add(siamese_net.layers[5].layers[i])

pred_1 = testmod_1.predict(xx_train[:,0,:,:,None])
pred_2 = testmod_2.predict(xx_train[:,0,:,:,None])
pred_3 = testmod_3.predict(xx_train[:,0,:,:,None])

# check if the weights in the siamese are really the same in all the three leg of the model:
a1 = np.linalg.norm(siamese_net.layers[3].layers[13].get_weights()[0])
a2 = np.linalg.norm(siamese_net.layers[4].layers[13].get_weights()[0])
a3 = np.linalg.norm(siamese_net.layers[5].layers[13].get_weights()[0])
print([siamese_net.layers[x].name for x in range(3,6)])
print([siamese_net.layers[x].layers[13].name for x in range(3,6)])
print(a1,a2,a3)
print('\nIn the models made out the layers take from the siamese model:')
a1 = np.linalg.norm(testmod_1.layers[14].get_weights()[0])
a2 = np.linalg.norm(testmod_2.layers[14].get_weights()[0])
a3 = np.linalg.norm(testmod_3.layers[14].get_weights()[0])
print(testmod_1.layers[14].name,testmod_2.layers[14].name,testmod_3.layers[14].name)
print(a1,a2,a3)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_13 (InputLayer)        (None, 60, 128, 1)        0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 60, 128, 4)        256       
_________________________________________________________________
batch_normalization_31 (Batc (None, 60, 128, 4)        16        
_________________________________________________________________
depthwise_conv2d_11 (Depthwi (None, 1, 128, 8)         480       
_________________________________________________________________
batch_normalization_32 (Batc (None, 1, 128, 8)         32        
_________________________________________________________________
activation_21 (Activation)   (None, 1, 128, 8)         0         
_________________________________________________________________
average_pooling2d_21 (Averag (None, 1, 32, 8)          0         
__________

In [68]:
xx_train[None,0,0,:,:,None].shape

(1, 60, 128, 1)

In [69]:
print(yy_test[0])
pred_1 = testmod_1.predict(xx_train[None, 0,0,:,:,None])

tt = 100
labs1 = []
labs0 = []
print(yy_test[tt])
pred_2 = testmod_1.predict(xx_train[None,tt,0,:,:,None])

for i in range(len(xx_train)):
    pred_2 = testmod_1.predict(xx_train[None,i,0,:,:,None])
    if np.linalg.norm(yy_train[i] - 1) == 0:
        labs1.append(i)
    else:
        labs0.append(i)
    print('result:','\t', i,'\t', yy_train[i],'\t', np.linalg.norm(pred_1-pred_2))

0.0
0.0
result: 	 0 	 1.0 	 0.0
result: 	 1 	 0.0 	 0.0515698
result: 	 2 	 1.0 	 0.0625995
result: 	 3 	 0.0 	 0.387477
result: 	 4 	 1.0 	 0.274205
result: 	 5 	 1.0 	 0.299221
result: 	 6 	 1.0 	 0.566591
result: 	 7 	 0.0 	 0.237894
result: 	 8 	 1.0 	 0.515248
result: 	 9 	 1.0 	 0.48981
result: 	 10 	 0.0 	 0.776937
result: 	 11 	 0.0 	 0.127648
result: 	 12 	 1.0 	 0.31852
result: 	 13 	 0.0 	 0.0748979
result: 	 14 	 0.0 	 0.156602
result: 	 15 	 0.0 	 0.600319
result: 	 16 	 0.0 	 0.686846
result: 	 17 	 1.0 	 0.0781404
result: 	 18 	 0.0 	 0.196362
result: 	 19 	 0.0 	 0.272538
result: 	 20 	 0.0 	 0.304549
result: 	 21 	 0.0 	 0.36488
result: 	 22 	 1.0 	 0.48981
result: 	 23 	 1.0 	 0.462892
result: 	 24 	 0.0 	 0.238791
result: 	 25 	 1.0 	 0.0233032
result: 	 26 	 0.0 	 0.279878
result: 	 27 	 0.0 	 0.721961
result: 	 28 	 0.0 	 0.0767553
result: 	 29 	 0.0 	 0.450621
result: 	 30 	 0.0 	 0.478036
result: 	 31 	 0.0 	 0.161759
result: 	 32 	 0.0 	 0.243627
result: 	 33 	 

result: 	 284 	 1.0 	 0.319343
result: 	 285 	 1.0 	 0.175058
result: 	 286 	 1.0 	 0.566591
result: 	 287 	 1.0 	 0.0
result: 	 288 	 0.0 	 0.11456
result: 	 289 	 0.0 	 0.254699
result: 	 290 	 1.0 	 0.0219877
result: 	 291 	 0.0 	 0.124393
result: 	 292 	 0.0 	 0.288481
result: 	 293 	 0.0 	 0.171171
result: 	 294 	 1.0 	 0.217958
result: 	 295 	 1.0 	 0.229719
result: 	 296 	 0.0 	 0.218822
result: 	 297 	 0.0 	 0.0824586
result: 	 298 	 0.0 	 0.289844
result: 	 299 	 1.0 	 0.316631
result: 	 300 	 0.0 	 0.108073
result: 	 301 	 1.0 	 0.0256016
result: 	 302 	 1.0 	 0.516536
result: 	 303 	 0.0 	 0.236628
result: 	 304 	 0.0 	 0.105843
result: 	 305 	 0.0 	 0.0353435
result: 	 306 	 0.0 	 0.260196
result: 	 307 	 0.0 	 0.289844
result: 	 308 	 0.0 	 0.0939713
result: 	 309 	 0.0 	 0.272538
result: 	 310 	 0.0 	 0.0102732
result: 	 311 	 0.0 	 0.36488
result: 	 312 	 0.0 	 0.0526275
result: 	 313 	 0.0 	 0.420134
result: 	 314 	 0.0 	 0.269203
result: 	 315 	 1.0 	 0.096921
result: 

result: 	 579 	 1.0 	 0.217599
result: 	 580 	 1.0 	 0.316605
result: 	 581 	 0.0 	 0.0130374
result: 	 582 	 0.0 	 0.641334
result: 	 583 	 0.0 	 0.244196
result: 	 584 	 0.0 	 0.339582
result: 	 585 	 0.0 	 0.253308
result: 	 586 	 0.0 	 0.289844
result: 	 587 	 0.0 	 0.450621
result: 	 588 	 1.0 	 0.0194682
result: 	 589 	 0.0 	 0.259815
result: 	 590 	 1.0 	 0.244661
result: 	 591 	 0.0 	 0.145091
result: 	 592 	 0.0 	 0.0857466
result: 	 593 	 1.0 	 0.796424
result: 	 594 	 0.0 	 0.395106
result: 	 595 	 1.0 	 0.264389
result: 	 596 	 1.0 	 0.115807
result: 	 597 	 0.0 	 0.278223
result: 	 598 	 1.0 	 0.344356
result: 	 599 	 0.0 	 0.210382
result: 	 600 	 0.0 	 0.411157
result: 	 601 	 1.0 	 0.231198
result: 	 602 	 1.0 	 0.141949
result: 	 603 	 1.0 	 0.368876
result: 	 604 	 0.0 	 0.143676
result: 	 605 	 1.0 	 0.627415
result: 	 606 	 1.0 	 0.162906
result: 	 607 	 1.0 	 0.473502
result: 	 608 	 0.0 	 0.0767553
result: 	 609 	 1.0 	 0.0386558
result: 	 610 	 0.0 	 0.339582
res

In [70]:
acc_1 = 0
for i in range(len(labs1)):
    acc_1 += np.linalg.norm(pred_1-testmod_1.predict(xx_train[None,labs1[i],0,:,:,None]))

acc_0 = 0
for i in range(len(labs0)):
    acc_0 += np.linalg.norm(pred_1-testmod_1.predict(xx_train[None,labs0[i],0,:,:,None]))

print('acc_1:', acc_1)
print('acc_0:', acc_0)

acc_1: 103.66836398
acc_0: 95.829120707


In [57]:
pred_2 = []
print(y_test[0])
pred_1 = testmod_1.predict(x_train[None,0,:,:,None])

for i in range(1,15):
    pred_2.append(np.linalg.norm(testmod_1.predict(x_train[None,i,:,:,None])))

# print('result:', np.linalg.norm(pred_1-pred_2))
print([np.linalg.norm(pred_1-pred_2[i]) for i in range(len(pred_2))])

[ 0.  1.]
[12.912468, 11.579817, 11.073095, 11.582479, 12.009239, 11.521235, 11.8072, 11.009918, 10.818402, 11.326399, 12.399729, 12.086484, 10.278913, 11.051972]


In [347]:
xx_train[:,0,:,:,None].shape

(539, 60, 128, 1)

In [326]:
xx_train[1][1].shape

(60, 128)

In [237]:
siamese_net.layers[3].layers[1].get_weights()

[array([ 1.01094961,  1.01399767,  1.01906908,  1.01613176], dtype=float32),
 array([  1.85610606e-05,   2.14835541e-06,   5.89442266e-07,
         -1.97443410e-06], dtype=float32),
 array([  5.95167112e-05,   2.45559531e-05,   5.38175373e-05,
          6.08851260e-05], dtype=float32),
 array([ 0.14509232,  0.11979525,  0.13534319,  0.13674594], dtype=float32)]

In [116]:
# def testtest():
#     #build convnet to use in each siamese 'leg'
#     input_shape = (64, 128, 1)
#     convnet = Sequential()
#     convnet.add(Conv2D(64,(10,10),activation='relu', input_shape=input_shape))
#     convnet.add(Conv2D(128,(7,7),activation='relu'))
#     convnet.add(Flatten())
#     convnet.add(Dense(2,activation="sigmoid"))
#     return convnet

In [111]:
# #build convnet to use in each siamese 'leg'
# input_shape = (64, 128, 1)
# convnet = Sequential()
# convnet.add(Conv2D(64,(10,10),activation='relu', input_shape=input_shape))
# convnet.add(Conv2D(128,(7,7),activation='relu'))
# convnet.add(Flatten())
# convnet.add(Dense(2,activation="sigmoid"))

In [53]:
def EEGNet(nb_classes, Chans = 64, Samples = 128, 
             dropoutRate = 0.25, kernLength = 64, F1 = 4, 
             D = 2, F2 = 8, dropoutType = 'Dropout'):
    """ Keras Implementation of EEGNet (https://arxiv.org/abs/1611.08024)

    Inputs:
        
      nb_classes      : int, number of classes to classify
      Chans, Samples  : number of channels and time points in the EEG data
      dropoutRate     : dropout fraction
      kernLength      : length of temporal convolution in first layer. We found
                        that setting this to be half the sampling rate worked
                        well in practice. For the SMR dataset in particular
                        since the data was high-passed at 4Hz we used a kernel
                        length of 32.     
      F1, F2          : number of temporal filters (F1) and number of pointwise
                        filters (F2) to learn. Default: F1 = 4, F2 = F1 * D. 
      D               : number of spatial filters to learn within each temporal
                        convolution. Default: D = 2
      dropoutType     : Either SpatialDropout2D or Dropout, passed as a string.

    """
    
    if dropoutType == 'SpatialDropout2D':
        dropoutType = SpatialDropout2D
    elif dropoutType == 'Dropout':
        dropoutType = Dropout
    else:
        raise ValueError('dropoutType must be one of SpatialDropout2D '
                         'or Dropout, passed as a string.')
    
    input1   = Input(shape = (1, Chans, Samples))

    ##################################################################
    block1       = Conv2D(F1, (1, kernLength), padding = 'same',
                                   input_shape = (1, Chans, Samples),
                                   use_bias = False, data_format='channels_first')(input1)
    block1       = BatchNormalization(axis = 1)(block1)
    block1       = DepthwiseConv2D((Chans, 1), use_bias = False, 
                                   depth_multiplier = D,
                                   depthwise_constraint = max_norm(1.), data_format='channels_first')(block1)
    block1       = BatchNormalization(axis = 1)(block1)
    block1       = Activation('elu')(block1)
    block1       = AveragePooling2D((1, 4), data_format='channels_first')(block1)
    block1       = dropoutType(dropoutRate)(block1)
    
    block2       = SeparableConv2D(F2, (1, 16),
                                   use_bias = False,
                                   padding = 'same',
                                   data_format='channels_first')(block1)
    block2       = BatchNormalization(axis = 1)(block2)
    block2       = Activation('elu')(block2)
    block2       = AveragePooling2D((1, 8), data_format='channels_first')(block2)
    block2       = dropoutType(dropoutRate)(block2)
        
    flatten      = Flatten(name = 'flatten')(block2)
    
    dense        = Dense(nb_classes, name = 'dense', 
                         kernel_constraint = max_norm(0.25))(flatten)
    softmax      = Activation('softmax', name = 'softmax')(dense)
    
    return Model(inputs=input1, outputs=softmax)

model  = EEGNet(nb_classes = 2,
                Chans = 60,
                Samples = 128,
                kernLength = 125)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_33 (InputLayer)        (None, 1, 60, 128)        0         
_________________________________________________________________
conv2d_26 (Conv2D)           (None, 4, 60, 128)        500       
_________________________________________________________________
batch_normalization_28 (Batc (None, 4, 60, 128)        16        
_________________________________________________________________
depthwise_conv2d_10 (Depthwi (None, 8, 1, 128)         480       
_________________________________________________________________
batch_normalization_29 (Batc (None, 8, 1, 128)         32        
_________________________________________________________________
activation_15 (Activation)   (None, 8, 1, 128)         0         
_________________________________________________________________
average_pooling2d_14 (Averag (None, 8, 1, 32)          0         
__________